In [3]:
iss =PrintIssue.objects.last()
iss.text

'Norges største studentavis\n | årgang 71, utgave 14 \n| www.universitas.no\n | onsdag 10. mai 2017\nGeologisk museum \n\nskulle bare ˜ytte:\nUtsatt for \ngrov rasisme \npå Blindern\nÅrevis med diskriminering:\n '

In [4]:
has_file = []
for pi in PrintIssue.objects.all():
    try:
        data = pi.get_page_text_content(2)
    except FileNotFoundError:
        pass
    else:
        has_file.append(pi)

In [60]:
import re
import pprint
positions = 'redaktør', 'desksjef', 'nyhetsleder', 'nettredaktør', 'fotosjef', 'debattredaktør', 'magasinredaktør'
pattern =  re.compile(r'^(?P<title>\w+):\s(?P<name>[-\w ]+)$', flags = re.M | re.I)
def find_positions(issue):
    content = issue.get_page_text_content(2,100)
    #print(content)
    matches = pattern.finditer(content)
    return {d['title']: d['name'] for d in [m.groupdict() for m in matches] if d['title'] in positions}

def add_to_stint(title, name, date):
    contributor = Contributor.objects.get(display_name=name)
    position, _ = Position.objects.get_or_create(title=title)
    defaults = {'start_date': date, 'end_date': date}
    stint, _ = Stint.objects.get_or_create(defaults=defaults, position=position, contributor=contributor)
    stint.end_date = max(stint.end_date, date)
    stint.start_date = min(stint.start_date, date)
    stint.save()
     
    
for issue in has_file:
    data = find_positions(issue)
    content = issue.get_page_text_content(2,100)
    print(issue)
    pprint.pprint(data)
    date = issue.issue.publication_date
    for title, name in data.items():
        add_to_stint(title, name, date)
    


/media/pdf/8f7b66a2e82a11e68c22f23c916e5154/universitas_2017-4.pdf
{'debattredaktør': 'Knut Arne Oseid',
 'desksjef': 'Lise Blekastad',
 'fotosjef': 'Adrian Nielsen',
 'magasinredaktør': 'Signe Rosenlund-Hauglid',
 'nettredaktør': 'Sondre Myhre',
 'nyhetsleder': 'Birk Tjeldflaat Helle',
 'redaktør': 'Torgeir Mortensen'}
/media/pdf/0aae96f6f8ab11e68c22f23c916e5154/universitas_2017-7.pdf
{'debattredaktør': 'Knut Arne Oseid',
 'desksjef': 'Lise Blekastad',
 'fotosjef': 'Adrian Nielsen',
 'magasinredaktør': 'Signe Rosenlund-Hauglid',
 'nettredaktør': 'Sondre Myhre',
 'nyhetsleder': 'Birk Tjeldflaat Helle',
 'redaktør': 'Torgeir Mortensen'}
/media/pdf/dc03a8b4d7e411e6b0dcf23c916e5154/universitas_2017-1.pdf
{'debattredaktør': 'Knut Arne Oseid',
 'desksjef': 'Lise Blekastad',
 'fotosjef': 'Adrian Nielsen',
 'magasinredaktør': 'Signe Rosenlund-Hauglid',
 'nettredaktør': 'Sondre Myhre',
 'nyhetsleder': 'Birk Tjeldflaat Helle',
 'redaktør': 'Torgeir Mortensen'}
/media/pdf/3d392c0edd2a11e6b0dcf23

In [61]:
def add_stints_from_bylines(title, credit):
    position, _ = Position.objects.get_or_create(title=title)
    for person in Contributor.objects.all():
        bylines = person.byline_set.filter(
            credit__contains=credit
        ).order_by(
            'story__publication_date'
        ).exclude(
            story__publication_date=None
        ).filter(
            story__story_type__section__title__in=['Nyhet', 'Kultur', 'Magasin']
        )
        if bylines.count() > 1:
            first_byline = bylines.first().story.publication_date.date()
            last_byline = bylines.last().story.publication_date.date()
            stint, created = Stint.objects.get_or_create(
                dict(start_date=first_byline, end_date=last_byline),
                contributor=person, position=position)
            if not created:
                stint.start_date = min(stint.start_date, first_byline)
                stint.end_date = max(stint.end_date, last_byline)
                stint.save()
            print(f'{stint} {bylines.count()}')
       
    
add_stints_from_bylines('journalist', 'text')
add_stints_from_bylines('fotograf', 'photo')


journalist Maria Reinertsen 11
journalist Hanna Skotheim 12
journalist Benedicte Tobiassen 11
journalist Anders R. Christensen 11
journalist Kjetil Nordengen 2
journalist Linda Johnsen Ørstavik 2
journalist Lise Grønskar 6
journalist Melike Leblebicioğlu 7
journalist Mathias Vedeler 4
journalist Hans Dalane-Hval 4
journalist Thorbjørn Kringlebotn Borlaug 4
journalist Magnus Lysberg 5
journalist Julie Nordby Egeland 4
journalist Eirik Omvik 14
journalist Anders Ballangrud 8
journalist Are W. Sandvik 13
journalist Emma Tollersrud 5
journalist Maria Terese Kittilsen 2
journalist Torgeir Mortensen 10
journalist Mads Randen 4
journalist Regine Stokstad 6
journalist Benedicte Tandsæther-Andersen 2
journalist Ingeborg Misje 5
journalist Brita Søreng 2
journalist Morten Oftedal Schwencke 2
journalist Sondre Myhre 4
journalist Ingeborg Grindheim Slinde 2
journalist Carina I. Hunshamar 2
journalist Mari Lund Haanshus 16
journalist Karsten Eig 2
journalist Kitty Byng 2
journalist Hedda Faldet Øst

journalist Ida Madsen Hestman 15
journalist Ingrid Eidsheim Daae 11
journalist Marit Aasmo Albrigtsen 2
journalist Anders Sundstøl Bjørkheim 3
journalist Eirik Billingsø Elvevold 9
journalist Vilde Sagstad Imeland 17
journalist Ingri Bergo 6
journalist Ida Elizabeth Torp Halvorsen 12
journalist Philip André Johannesborg 9
fotograf Henrik Evertsson 42
fotograf Silje Ramstad 30
fotograf Ketil Blom 151
fotograf Igor Nilsen 12
fotograf Hans Dalane-Hval 94
fotograf Birte Nystad Magnussen 22
fotograf Haakon Jamtli Kristiansen 3
fotograf Helene Kristensen 2
fotograf Sjur Stølen 7
fotograf Melisa Fajkovic 5
fotograf Henriette Dæhli 3
fotograf Øyvind Aukrust 2
fotograf Raisa Porsanger 4
fotograf Patrick da Silva Sæther 29
fotograf Mari Lund Haanshus 22
fotograf Christian Breidlid 3
fotograf Dorthe Karlsen 8
fotograf Audun Halaas 28
fotograf Xueqi Pang 5
fotograf Vilde Borse 3
fotograf Hanna Hjardar 2
fotograf Hanna Kristin Hjardar 3
fotograf Angelique Culvin 5
fotograf Hilde Raa 2
fotograf Hann

In [62]:
debatt_bylines = Byline.objects.filter(story__story_type__section__title='Debatt')
debatt_bylines.filter(credit='text').update(credit='by')

0

In [63]:
from django.utils import timezone
today = timezone.now().date()
six_months = timezone.timedelta(days=365//2)

for contributor in Contributor.objects.all():
    all_bylines = contributor.byline_set.exclude(story__publication_date=None)
    debatt_bylines = all_bylines.filter(story__story_type__section__title='Debatt')
    if all_bylines.count() == 0:
        print(contributor)
    elif all_bylines.count() == debatt_bylines.count():
        contributor.status = contributor.EXTERNAL
    else:
        latest = all_bylines.latest('story__publication_date').story.publication_date.date()
        if today - latest > six_months:
            contributor.status = contributor.RETIRED
        else:
            contributor.status = contributor.ACTIVE
    contributor.save()
    print(contributor, contributor.get_status_display())
    

Maria Reinertsen Slutta
Ingrid Os Ekstern
Jan Frich Ekstern
Caroline Rugeldal Slutta
Kristoffer Guttorm Camillo Pedersen Ekstern
Morten Nordby Ekstern
Hanna Skotheim Aktiv
Henrik Evertsson Slutta
Benedicte Tobiassen Aktiv
Siri Halland Nesse Ekstern
Anders R. Christensen Slutta
Kjetil Nordengen Slutta
Silje Ramstad Slutta
Ketil Blom Slutta
Linda Johnsen Ørstavik Slutta
Lise Grønskar Slutta
Melike Leblebicioğlu Slutta
Mathias Vedeler Slutta
Kirsty Johansen Slutta
Hallvard Østtveit Barbogen Slutta
Igor Nilsen Slutta
Hans Dalane-Hval Slutta
Thorbjørn Kringlebotn Borlaug Slutta
Birte Nystad Magnussen Slutta
Cecilia Johansen Eriksen Slutta
Haakon Jamtli Kristiansen Slutta
Magnus Lysberg Slutta
Julie Nordby Egeland Slutta
Ørjan Ryland Slutta
Helene Kristensen Slutta
Eirik Omvik Slutta
Anders Ballangrud Slutta
Are W. Sandvik Slutta
Emma Tollersrud Slutta
Adrien Vigier Ekstern
Emil Øversveen Slutta
Odd Vegard Paulsen Ekstern
Vetle T. Westlie Ekstern
Tina Årikstad Sørensen Ekstern
Kjartan Hverve

M. Havstein Slutta
Hagbard Celine Ekstern
Karin Yrvin Ekstern
Torbjørn Folstad Ekstern
Ole Skogedal Ekstern
Kari Benz Slutta
Gunn Elisabeth Birkelund Ekstern
Bård Hermansen Ekstern
Lucas H. Weldeghebriel Slutta
Anders Moe Ekstern
Even Hovdhaugen Ekstern
Jo Haugen Ekstern
Audun Automat Slutta
Kansas Slutta
Thomas Eliassen Slutta
Vennlig Hilsen Ekstern
Olav Torheim Ekstern
Jan Gunnar Aschim Ekstern
Hanne Andrea Kraugerud Ekstern
Per Kristian Nielsen Ekstern
Bjørn Larsen Ekstern
Lars Magnus Bergh Ekstern
Dag Hjermann (Oops Ekstern
Hans Prydz Ekstern
Hayastan Slutta
Erik Lunde Ekstern
Hans–Jul Mikkelsen Ekstern
Pål Henriksen Ekstern
Erik Lundesgaard Ekstern
Johanne Thornes Ekstern
Emmanuel Slutta
Marianne Store Søndmør Slutta
Hege Prestby Andersen Ekstern
Anne Berit L. Grøtterud Ekstern
Erik Møller Solheim Slutta
Heidi Røsok Slutta
Lars Henie Barstad Slutta
Mona Tandsether Slutta
Pål Meland Ekstern
Lasse Thue Slutta
Dag Stian Husby Slutta
Sunniva Trengereid Slutta
Marianne Johansen Slutta


Terje Hoel Ekstern
Iman Winkelman Ekstern
Høyre Ekstern
Lene Hoem Slutta
Danem Johanson Slutta
Pv + Krn Slutta
Vl&Jh Slutta
Tony Burner Ekstern
Kristian Slutta
The Main Character Slutta
Kristian Gundersen Ekstern
Jens Å. R. Haga Ekstern
Kim Rand–Hendriksen Ekstern
Skribent Slutta
La Linea Slutta
Benjamin Endré Larsen Ekstern
Iselin Rud–Goksøyr Ekstern
Art Vandalay Slutta
Mr.Wrong Slutta
Greg Mccartan Ekstern
Atle Qviller Ekstern
Professor Knut Sydsæter Ekstern
Morten Berg Ekstern
Krister Øie Braute Ekstern
Anne Viken Ï Peter Vollset Slutta
Silje Eide Slutta
Tage Wester Slutta
Jan Eivind Myhre Ekstern
Knut Hammersmark Ekstern
Anders Mølster Galaasen Slutta
Eivind Sørlie Slutta
Ellen Margrete Ceeberg Slutta
Frk. Spillegal Slutta
Eva–Kristin Urestad Pedersen Ekstern
Linn Beate Kaald Thoresen Ekstern
Harald Fagerheim Bugge Slutta
Kristin Fredriksen Slutta
Tor Audun Kveim Gram Slutta
Joakimm Steegen Slutta
Diderik Lund Ekstern
Petra Oline Tommelise Slutta
Juristforeningen Ved Marie Meling E

Benedikte Skarvik Slutta
Øystein Djupedal Kunnskapsminister Ekstern
Robin Sande Ekstern
Lene C. Westgaard Studentparlamentets Arbeidsutval Ekstern
Andreas Pahle Ekstern
Prinsessa Som Ingen Ville Målbinde Slutta
Anita Hillestad Slutta
Jonas Holmqvist Leder Saih Ekstern
Christine Oppegaard Ekstern
Cato Hernes Jensen Ekstern
Kunnskapsminister Øystein Djupedal Ekstern
Mari Tangvall Eggen Slutta
Filip Roshauw Slutta
Ingrid Solhaug Slutta
Marte Skjesol Slutta
Ingrid Lomelde Slutta
- Slutta
Professor Bjarne Hodne Ekstern
Med God Tru Slutta
Rune Kinn Anjum Slutta
Damir Cvetojevic Slutta
Ali Hagi Slutta
Joanna Rzadkowska Slutta
Ståle Wilhelmsen Ekstern
Eystein Halle Ekstern
Afsoon Eshghipour Ekstern
Johan Stenshorne Ekstern
Sebastian Wemmerløv Slutta
Vic Tim Ov Crime Slutta
Kjartan Almenning Landsstyremedlem I Nsu Ekstern
Edvin Søvik Ekstern
Marie Nyhuus Ekstern
Tideli Bom Slutta
Jamal Mohsin Slutta
Ola Tellesbø Ekstern
Per Øyvind Voie Ekstern
På Vegne Av Studentkjellerne Ekstern
Dns Ekstern
Jo

Haakon Breien Benestad Ekstern
Robert J. Quinn Ekstern
Liberal Liste Ekstern
Anders C. Heli Ekstern
Kjellerutvalget Ekstern
Det Norske Studentersamfund Ekstern
Karoline Bekkoset Ekstern
Dafina Percuku Asani Ekstern
Kroatisk Ekstern
Serbisk Kulturkunnskap Ekstern
Historieløs Slutta
Jostein Riiser Kristiansen Ekstern
Peter Holme Ekstern
Månebarnet Slutta
Illustrasjon: Mari Lund Haanshus Slutta
Universitas´ Juleølpanel Slutta
Professor Øystein Sørensen Slutta
Andreas Hardhaug Olsen Ekstern
Cecilie Endresen Ekstern
Chalak Kaveh Ekstern
Ingrid Tungen Ekstern
Tom Olstad Ekstern
Anna Werenskjold Slutta
Øystein Aldridge Slutta
Petrine Nyhus Slutta
Trygve Thorson Ekstern
Kristin Aursand Ekstern
Lennart A. Lie-Havstein Ekstern
Kristian Mollestad Ekstern
Tor Egil Førland Ekstern
Øystein Idsø Viken Ekstern
Christina Egeland I Velferdstingets Arbeidsutvalg Ekstern
Jenny Kathrine Lorentzen Ekstern
Barne- Og Likestillingsminister Anniken Huitfeldt Ekstern
Onkel Bestemor Slutta
Marte Refsdal Slutta
Ta

Stephen Hutton Slutta
Tidligere Bachelorstudenter I Internasjonale Studi Ekstern
Universitas’ Sommerøltestpanel Slutta
Mette Hanekamhaug Ekstern
Representant I Kirke- Ekstern
Elin Nesje Vestli Ekstern
Maria Svendsen Ekstern
Aina Stenersen Ekstern
Helene Bjerke Fredheim Ekstern
) Slutta
Stine Elise Christensen Ekstern
Melike Leblebicio?Lu Ketil Blom Slutta
Helge Holgersen Ekstern
Ståle Pallesen Ekstern
Ketil Mathiassen Ekstern
Fanny Duckert Ekstern
Siri Nerbø Ekstern
Pål Buset Ekstern
Bendik Løve Slutta
Mikael Lunde Slutta
Lektorprogrammets Programutvalg Ekstern
Erlend Bergsås Mikkelsen Ekstern
Tia Karlsen Skjalg Bøhmer Vold Slutta
Anne Høyer Slutta
Skjalg Vold Slutta
Heidi Bade Slutta
Stella Heieren Slutta
Aurora Hannisdal Slutta
Tonje Næss Slutta
Per Harald Borgen Slutta
Hf-Lista Ekstern
Ole Martin Moen Ekstern
Oscar Dybedahl Ekstern
Åse Holte Slutta
Maria Terray Brantenberg Slutta
Maria Torkilsen Horvei Slutta
Tia Karlsen Slutta
Aksel Hagen Ekstern
Isis Sunniva Deleon Ekstern
Kaja Sa

Christoffer Gundersen Slutta
Stine Stenhaug Slutta
Maiken Alm Slutta
Kenneth Wangen Slutta
Kaja Storrøsten Aktiv
Øyvind Berdal Ekstern
Knut Hermundstad Aukrust Slutta
Mette Vabø Ekstern
Lene Christensen Slutta
Nils Magne Killingberg Ekstern
Ådne Hindenes Ekstern
Eirik Sti Ekstern
Celestina Da Silva Ekstern
Janni Kalafatis Slutta
Eivind Rindal Ekstern
Gunnar W. Schade Ekstern
Shkurta Olluri Ekstern
Silje Christine Andersen Ekstern
Martin Tveten Ekstern
Jørn Kløvfjell Mjelva Ekstern
Anita Tøien Johansen Ekstern
Ole Henrik Ellestad Ekstern
Christian A. Drevon Ekstern
Øyvind Foss  Ekstern
Bjørn Kristian Danbolt Ekstern
Eirik Gjerløw Ekstern
Maksut Jan Ekstern
Olle Törnquist Ekstern
Ragnhild Rath Moritz-Olsen Ekstern
Mari Lund Wictorsen Slutta
Stian Berger Røsland Ekstern
Elisabeth Løland Ekstern
Line Ørnes Slutta
Ola Magnusson Rydje Ekstern
Vegard G. Wennesland Ekstern
Benedicte Elisabeth Bjerknes Slutta
Isabel Ånestad Ekstern
Sio Helse Ekstern
Aili Røtterud Løchen Slutta
Sverre Olav Trovi

Anders Nordmo Kvammen Slutta
Helena Nielsen Slutta
Sten Ånnerud Ekstern
Sindre Granly Meldalen Slutta
Anders Imenes Slutta
Marius Nergård Pettersen Slutta
Marianne Torp Ekstern
Peking Duck Soup Slutta
Thomas A Syvertsen Ekstern
Anne Leifsdatter Grønlund Slutta
June Torgersen Slutta
Professor Vidar L. Haanes Ekstern
Kristina Kvamme Nygård Ekstern
Olav Øye Ekstern
Børt Ølgve Slutta
Ellen Paalgaard Slutta
Mai-Bente Paulsen Slutta
Liv Rønnaug Lilleåsen Slutta
Heidi T. Harto Slutta
Ingrid K. Fredriksen Slutta
Andreas Falkenberg Slutta
Marit Austvik Letnes Slutta
The Womanizer Slutta
Bente Wabakken Hognestad Ekstern
Senter For Tverrfaglig Kjønnsforskning Ekstern
Ingrid H. F. Brubaker Slutta
Axel M. K. Hærland Slutta
Christel Gundelach Ekstern
Universitas’ Matpanel Slutta
Scandinavian Urban Studies Term Ekstern
Universitas’ Matpanel. Foto: Ketil Blom Slutta
Translation By Ingrid F. Brubaker Slutta
Kristoffer Hatteland Endresen Slutta
Tonje Thielsen Slutta
Birgit Skarstein Ekstern
Øystein Grøn

In [72]:
# Fjern ubrukte artikkeltyper

for st in StoryType.objects.all():
    if st.story_set.count() == 0:
        print(st)
        st.delete()
        
for sec in Section.objects.all():
    story_types = sec.storytype_set.all()
    if story_types.count() == 0:
        sec.delete()
    print(sec)
    for story_type in story_types:
        print('   ', story_type, story_type.story_set.count())

Nyheter
    Nett 884
    News 700
    Utenriks 451
    Nyhet 4565
Magasin
    Camillas Kollektion 29
    Reisebilag 18
    Velkomstbilag 62
    Magasin 963
    Petit 273
Kultur
    Akademia 21
    Kultur 2105
    Kulturnotis 2
Debatt
    SMS 192
    Debatt 38
    Leserbrev 2198
    Kronikk 488
Baksiden
    Ad Notam 288
    Vi spør 322
Side 2
    Øyeblikket 125
    Kommentar 572
    Leder 507
Anmeldelser
    Hva skjer 167
    Anmeldelse 3279
Feature
    Reportasje 20
    Studietid 955
    Min studietid 342
Annet
    Universitas Melder 23
    Tassen 9
    Quiz 5
